# Determining the Best Value Ski Hotels in Europe
Ben Kagan  
CMSC320  
Maksym Morawski  
December 16th, 2022

## Introduction

Europe contains a vast number of ski resorts, ranging from beginner-friendly resorts suitable for a day-trip to mega resorts that have hosted international competitions. Many ski resorts play host to numerous hotels that market its proximity to the resort as a luxury. A wide variety of factors can help determine the value of one ski hotel over another. In addition to the standard factors including, but not limited to, price, customer reviews, and hotel quality, the value of a ski hotel heavily depends on factor such as the proximity to the ski resort and the ski resort's overall quality to name a few.

In this tutorial, we will analyze data on over 400 ski hotels in Europe by performing data exploration and subsequently regression analysis and value determination.

## Data Collection

The data obtained for this tutorial can be found on the following Kaggle page: [Skiing Hotels](https://www.kaggle.com/datasets/jacklacey/skiing-hotels).

To start, we will import all the libraries used in this tutorial:

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [3]:
hotels_df = pd.read_csv("ski_hotels.csv")
hotels_df = hotels_df.iloc[: , 1:]
hotels_df

,country,resort,hotel,price (£),distance_from_lift_(m),altitude (m),totalPiste (km),totalLifts,gondolas,chairlifts,...,blacks,totalRuns,link,sleeps,decSnowLow2020(cm),decSnowHigh2020(cm),janSnowLow2020(cm),janSnowHigh2020(cm),febSnowLow2020(cm),febSnowHigh2020(cm)
0,italy,bardonecchia,residence-tabor,550,unknown,1312,140,23,0,9,...,1.0,42.0,https://www.igluski.com/ski-resorts/italy/bard...,unknown,70,170,60,200,47,177
1,italy,bardonecchia,residence-villa-frejus,561,unknown,1312,140,23,0,9,...,1.0,42.0,https://www.igluski.com/ski-resorts/italy/bard...,unknown,70,170,60,200,47,177
2,bulgaria,bansko,hotel-mura,566,1100,935,70,24,1,7,...,1.0,14.0,https://www.igluski.com/ski-resorts/bulgaria/b...,140,0,70,5,75,5,77
3,bulgaria,borovets,hotel-samokov,574,75,1390,58,18,1,4,...,1.0,15.0,https://www.igluski.com/ski-resorts/bulgaria/b...,590,0,0,160,200,194,246
4,bulgaria,bansko,hotel-lion---bansko,596,800,935,70,24,1,7,...,1.0,14.0,https://www.igluski.com/ski-resorts/bulgaria/b...,386,0,70,5,75,5,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,france,val-thorens,hotel-fitz-roy,2216,unknown,2300,600,183,40,69,...,32.0,321.0,https://www.igluski.com/ski-resorts/france/val...,112,98,121,215,267,207,269
403,austria,ischgl,hotel-fliana,2258,unknown,1400,230,48,9,27,...,15.0,98.0,https://www.igluski.com/ski-resorts/austria/is...,30,20,70,50,120,63,131
404,austria,ischgl,hotel-elisabeth,2420,unknown,1400,230,48,9,27,...,15.0,98.0,https://www.igluski.com/ski-resorts/austria/is...,110,20,70,50,120,63,131
405,austria,ischgl,hotel-trofana-royal,2484,unknown,1400,230,48,9,27,...,15.0,98.0,https://www.igluski.com/ski-resorts/austria/is...,164,20,70,50,120,63,131


For convenience's sake, I will add a column labelled 'price ($)' to display the nightly price of the hotel in USD as opposed to in British pounds. As of December 14th, 2022. The conversion rate from 1 Pound sterling to United States Dollar is 1.23.

In [4]:
hotels_df['price ($)'] = hotels_df['price (£)'].apply(lambda x: x*1.23)

### Column Descriptions
Next, we need to determine the meaning of every column in our dataset in order further explore the data. Provided below is a list giving a description on each column of the dataset:
  
 - country: The country of the hotel and ski resort
 - resort: The name of the ski resort in which the hotel is located
 - hotel: The name of the hotel
 - price (£): The nightly price for a room in the hotel in British pounds
 - distance_from_lift_(m): The distance from the hotel to the nearest ski resort lift in meters
 - altitude (m): The altitude of the hotel in meters
 - totalPiste (km): The total distance of regularly maintained, skiable on the resort, in kilometers
 - totalLifts: The total number of lifts in the ski resort
 - gondolas: The number of gondolas in the ski resort
 - chairLifts: The number of chair lifts in the ski resort
 - dragLifts: The number of drag lifts in the ski resort
 - blues: The number of blue-graded slopes in the ski resort
 - reds: The number of red-graded slopes in the ski resort
 - blacks: The number of black-graded slopes in the ski resort
 - totalRuns: The total number of slopes in the ski resort
 - link: the URL to the hotel's website
 - sleeps: the total occupancy of the hotel
 - decSnowLow2020(cm): The smallest snowfall received, in centimeters, from the month of December 2020 at the resort
 - decSnowHigh2020(cm): The largest snowfall received, in centimeters, from the month of December 2020 at the resort
 - janSnowLow2020(cm): The smallest snowfall received, in centimeters, from the month of January 2020 at the resort
 - janSnowHigh2020(cm): The largest snowfall received, in centimeters, from the month of January 2020 at the resort
 - febSnowLow2020(cm): The smallest snowfall received, in centimeters, from the month of February 2020 at the resort
 - febSnowHigh2020(cm): The largest snowfall received, in centimeters, from the month of February 2020 at the resort
 - price ($): The nightly price for a room in the hotel in USD

### Web Scraping

Upon further inspection, it appears that the provided dataset is missing some vital data about the ski hotels, such as customer reviews. To remedy this, we will go ahead and scrape the Google ratings, on a 5-star scale, for each hotel in the dataset.

To achieve this, we will use the Selenium package to run a Chrome WebDriver to search into google the name of the hotel with the resort name. We will then parse the DOM Selenium's built-in `find_element()` function as well as XPath to retrieve the html element containing the star rating and the number of reviews. If a particular hotel search provides an abnormal result, the field is filled as unknown and consequently retrieved manually.

In [5]:
def scrape_google_results(driver, hotel, resort, tag):
    url = f"https://www.google.com/search?q={hotel}+{resort}"
    driver.get(url)
    time.sleep(3)
    try:
        element = driver.find_element(By.XPATH, '//div[@id="rhs"]/div/div/div[@class="I6TXqe"]/div/div[1]/div/div[2]/div[@class="wDYxhc"][1]/div/div').find_elements(By.TAG_NAME, tag)[0]
    except:
        return 'unknown'
    return element.text

def get_rating(driver, hotel, resort):
    return scrape_google_results(driver, hotel, resort, 'span')

def get_num_reviews(driver, hotel, resort):
    return scrape_google_results(driver, hotel, resort, 'a').split(" ")[0]
    
driver = webdriver.Chrome()

hotels_df['ratings'] = hotels_df.apply(lambda x: get_rating(driver, x.hotel, x.resort), axis=1)
hotels_df['num_reviews'] = hotels_df.apply(lambda x: get_num_reviews(driver, x.hotel, x.resort), axis=1)

### Missing/Unknown Data

Let's examine the data and determine how many rows contain unknown values. We will use this to decide what needs to be researched or dropped.

In [6]:
hotels_df = hotels_df.replace('unknown', np.nan)
hotels_df = hotels_df.astype({'price (£)': 'float64',
                              'distance_from_lift_(m)': 'float64',
                              'altitude (m)': 'float64',
                              'totalPiste (km)': 'float64',
                              'blues': 'int64',
                              'reds': 'int64',
                              'blacks': 'int64',
                              'totalRuns': 'int64',
                              'sleeps': 'int64',
                              'decSnowLow2020(cm)': 'float64',
                              'decSnowHigh2020(cm)': 'float64',
                              'janSnowLow2020(cm)': 'float64',
                              'janSnowHigh2020(cm)': 'float64',
                              'febSnowLow2020(cm)': 'float64',
                              'febSnowHigh2020(cm)': 'float64',
                              'ratings': 'float64',
                              'num_reviews': 'int64'}, errors='ignore')

The following line displays all hotels that had an abnormal hotel search, resulting in a missing rating and review count:

In [7]:
hotels_df[hotels_df[['ratings', 'num_reviews']].isna().any(axis=1)]

,country,resort,hotel,price (£),distance_from_lift_(m),altitude (m),totalPiste (km),totalLifts,gondolas,chairlifts,...,sleeps,decSnowLow2020(cm),decSnowHigh2020(cm),janSnowLow2020(cm),janSnowHigh2020(cm),febSnowLow2020(cm),febSnowHigh2020(cm),price ($),ratings,num_reviews
8,andorra,arinsal,hotel-xalet-verdu,606.0,300.0,1550.0,63.0,30,2,11,...,104,30.0,60.0,130.0,180.0,120.0,170.0,745.38,NaN,NaN
19,italy,sauze-d-oulx,hotel-clotes,692.0,100.0,1503.0,400.0,89,0,35,...,NaN,150.0,220.0,35.0,300.0,96.0,280.0,851.16,,NaN
28,austria,bad-hofgastein,apartments-aurora,704.0,NaN,870.0,208.0,49,11,19,...,NaN,0.0,100.0,48.0,155.0,45.0,176.0,865.92,NaN,NaN
39,finland,pyha,pyha-suites,737.0,NaN,280.0,80.0,9,0,0,...,4,40.0,50.0,60.0,70.0,64.0,74.0,906.51,NaN,NaN
55,austria,niederau-oberau,hotel-austria,761.0,NaN,826.0,48.0,47,7,8,...,94,0.0,0.0,50.0,106.0,48.0,114.0,936.03,NaN,NaN
63,bulgaria,pamporovo,hotel-orlovetz,772.0,3000.0,1650.0,18.0,19,0,6,...,210,0.0,0.0,60.0,85.0,166.0,170.0,949.56,NaN,NaN
70,austria,kitzbuhel,hotel-bruggerhof,783.0,2000.0,760.0,185.0,61,11,27,...,56,NaN,NaN,NaN,NaN,NaN,NaN,963.09,NaN,NaN
73,finland,yllas,yllas-saaga-spa-hotel,785.0,NaN,719.0,0.0,29,1,1,...,168,40.0,65.0,85.0,85.0,85.0,85.0,965.55,NaN,NaN
78,italy,courmayeur,hotel-courmayeur,795.0,NaN,1224.0,100.0,16,5,8,...,70,40.0,100.0,40.0,100.0,41.0,117.0,977.85,NaN,NaN
117,austria,zell-am-see,garden-hotel-daxer,861.0,NaN,758.0,136.0,55,13,18,...,NaN,0.0,58.0,43.0,147.0,46.0,150.0,1059.03,4.8,NaN


We will go ahead and manually Google search for each hotel listed above to retrieve their Google star rating and number of reviews.

In [10]:
hotels_df.at[8,'ratings'], hotels_df.at[8,'num_reviews'] = 4.3, 931
hotels_df.at[19,'ratings'], hotels_df.at[19,'num_reviews'] = 4.2, 32
hotels_df.at[28,'ratings'], hotels_df.at[28,'num_reviews'] = 4.3, 25
hotels_df.at[39,'ratings'], hotels_df.at[39,'num_reviews'] = 4.3, 24
hotels_df.at[55,'ratings'], hotels_df.at[55,'num_reviews'] = 3.8, 227
hotels_df.at[63,'ratings'], hotels_df.at[63,'num_reviews'] = np.nan, np.nan # no hotel listing on Google
hotels_df.at[70,'ratings'], hotels_df.at[70,'num_reviews'] = 4.2, 122
hotels_df.at[73,'ratings'], hotels_df.at[73,'num_reviews'] = 4.3, 826
hotels_df.at[117,'ratings'], hotels_df.at[117,'num_reviews'] = 4.8, 390
hotels_df.at[126,'ratings'], hotels_df.at[126,'num_reviews'] = 4.1, 61
hotels_df.at[159,'ratings'], hotels_df.at[159,'num_reviews'] = 4.6, 245
hotels_df.at[161,'ratings'], hotels_df.at[161,'num_reviews'] = 4.2, 520
hotels_df.at[171,'ratings'], hotels_df.at[171,'num_reviews'] = 4.7, 64
hotels_df.at[176,'ratings'], hotels_df.at[176,'num_reviews'] = 4.6, 122
hotels_df.at[190,'ratings'], hotels_df.at[190,'num_reviews'] = 4.6, 86
hotels_df.at[212,'ratings'], hotels_df.at[212,'num_reviews'] = 4.5, 557
hotels_df.at[217,'ratings'], hotels_df.at[217,'num_reviews'] = np.nan, np.nan # no reviews posted for Google listing
hotels_df.at[220,'ratings'], hotels_df.at[220,'num_reviews'] = np.nan, np.nan # no reviews posted for Google listing
hotels_df.at[228,'ratings'], hotels_df.at[228,'num_reviews'] = 4.2, 200
hotels_df.at[242,'ratings'], hotels_df.at[242,'num_reviews'] = 4.5, 73
hotels_df.at[249,'ratings'], hotels_df.at[249,'num_reviews'] = 4.0, 1
hotels_df.at[253,'ratings'], hotels_df.at[253,'num_reviews'] = 3.3, 282
hotels_df.at[269,'ratings'], hotels_df.at[269,'num_reviews'] = 4.3, 81
hotels_df.at[279,'ratings'], hotels_df.at[279,'num_reviews'] = 3.9, 151
hotels_df.at[282,'ratings'], hotels_df.at[282,'num_reviews'] = np.nan, np.nan # no reviews posted for Google listing
hotels_df.at[295,'ratings'], hotels_df.at[295,'num_reviews'] = 4.0, 67
hotels_df.at[326,'ratings'], hotels_df.at[326,'num_reviews'] = np.nan, np.nan # no reviews posted for Google listing
hotels_df.at[349,'ratings'], hotels_df.at[349,'num_reviews'] = 4.7, 71

We will drop the entries with no reviews, as that predictor is vital for our assessments:

In [11]:
hotels_df = hotels_df.dropna(subset=['ratings', 'num_reviews'])

One column of note that contains missing data is the `distance_from_lift_(m)` column. Despite its importance on the value of a ski hotel, we have decided to not heavily consider this metric in our assessments since over half of the entries are missing this value.

Due to the computational intensity of the web scraping in order to complete the dataset. I will save the DataFrame to a fresh .csv file in the event of data loss:

In [14]:
hotels_df.to_csv('ski_hotels_complete.csv' ,index=False)

## Data Exploration

Now that we have fully acquired the data on European Ski Hotels, we will preform some exploration to examine the properties of the data.